# 過学習

学習済みモデルの評価や過学習という課題、そしてその対策について。


---

## モデルの評価

学習済みモデルをどう評価しようかという話。

基本的にはモデルの予測値が正解とどれだけ近いかを評価する。その評価指標は色々あり、例えば回帰だったら平均二乗誤差とか、それを正規化した決定係数とか、また分類だったら正解率とか。

これらの指標を求めるためにはモデルへの入力と正解が適当な数必要になるが、それらはどこから得ると良いだろう。学習させたデータから適当に持ってくると良いと思うかもしれないが、それではモデルの**汎化性能**を測れない。学習時にモデルが見なかった「未知のデータ」に対してどのようなパフォーマンスを発揮するかが測れない。

そこで、評価指標の算出には学習時に使わなかったデータを用いるようにする。用意したデータの一部は学習に使用せず残しておき、学習後にそのデータを使って評価指標を算出する。学習に使用するデータは**訓練データ**、評価に使用するデータは**テストデータ**と呼ぶ。

やってみよう。

In [1]:
# サンプルデータの読み込み
from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y=True)

これを訓練データとテストデータに分ける。

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
len(X_train), len(X_test)

(331, 111)

ランダムで3:1に分けられた。ここから訓練データ（`X_train, y_train`）を学習させ、テストデータ（`X_test, y_test`）で評価する。

In [3]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test) # 決定係数

0.5885867966200098

これで汎化性能を考慮した評価ができた。